In [1]:
import torch
import sys
import os
from torchvision.datasets.folder import pil_loader
from torchvision.transforms import Normalize, Compose, RandomHorizontalFlip, RandomRotation, Resize, ToTensor, ToPILImage
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [2]:
device = torch.device('cuda')

In [3]:
device

device(type='cuda')

In [4]:
path = os.path.abspath(os.path.join('..'))

In [5]:
if path not in sys.path:
    sys.path.append(path+"/mish_cuda/")

In [6]:
from mish_cuda.mish import MishCuda

In [5]:
if path not in sys.path:
    sys.path.append(path)

In [6]:
sys.path

['/home/adtp/github/adtp/ScaledYOLOv4/notebooks',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python38.zip',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8/lib-dynload',
 '',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8/site-packages',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8/site-packages/IPython/extensions',
 '/home/adtp/.ipython',
 '/home/adtp/github/adtp/ScaledYOLOv4']

In [9]:
import models

### Turn model checkpoint into state_dict
useful to use our Mish instead of MishCuda

In [13]:
save_scaled = torch.load('yolov4-p7.pt', map_location=device)

In [14]:
weight_scaled = save_scaled['model'].state_dict()

In [15]:
torch.save(weight_scaled, 'yolov4-p7.pth')

In [16]:
weights_scaled_load = torch.load('yolov4-p7.pth')

In [19]:
save_scaled['model'].load_state_dict(weights_scaled_load)

SyntaxError: invalid syntax (<ipython-input-19-650dd39b2daf>, line 1)

In [ ]:
python detect.py --weights ../weights/yolov4-p7.pth --cfg data/coco.yaml --source ../person.jpg

## Load model

In [6]:
from models.yolo import Model

In [7]:
net = Model(cfg='../models/yolov4-p6.yaml', ch=3, nc=80)


                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     19904  models.common.BottleneckCSP             [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    161152  models.common.BottleneckCSP             [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1   2614016  models.common.BottleneckCSP             [256, 256, 15]                
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1  10438144  

Conv(
  (conv): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): Mish()
)
BottleneckCSP(
  (cv1): Conv(
    (conv): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): Mish()
  )
  (cv2): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (cv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (cv4): Conv(
    (conv): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): Mish()
  )
  (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): Mish()
  (m): Sequential(
    (0): Bottleneck(
      (cv1): Conv(
        (conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 

Conv(
  (conv): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): Mish()
)
Detect(
  (m): ModuleList(
    (0): Conv2d(256, 340, kernel_size=(1, 1), stride=(1, 1))
    (1): Conv2d(512, 340, kernel_size=(1, 1), stride=(1, 1))
    (2): Conv2d(1024, 340, kernel_size=(1, 1), stride=(1, 1))
    (3): Conv2d(1024, 340, kernel_size=(1, 1), stride=(1, 1))
  )
)
Model Summary: 599 layers, 1.27591e+08 parameters, 1.27591e+08 gradients



In [8]:
weights_scaled = torch.load('/home/adtp/github/adtp/weights/yolov4-p6.pth')

In [9]:
net.load_state_dict(weights_scaled)

<All keys matched successfully>

## Experiment inference

In [10]:
net.to(device)
net.eval();

In [13]:
# img_test = ToTensor()(Resize((416, 416))(pil_loader('bradley-cooper.png'))).unsqueeze_(0)
# img_test = ToTensor()(Resize((800, 1280))(pil_loader('street.jpg'))).unsqueeze_(0)
img_test = ToTensor()(Resize((640, 640))(pil_loader('/home/adtp/github/adtp/street.jpg'))).unsqueeze_(0)

In [14]:
pred = net(img_test.to(device))

Conv(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): Mish()
)
Conv(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): Mish()
)
BottleneckCSP(
  (cv1): Conv(
    (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): Mish()
  )
  (cv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (cv3): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (cv4): Conv(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): Mish()
  )
  (bn): BatchNorm2d(64, eps=0.001, mom

In [ ]:
pred[0].shape

In [ ]:
torch.argmax(pred[0][:, :, 5:], dim=2)